# 2. 분석 후 맛집 추천

## 1) 데이터 전처리
- 결측치 처리
- 행 분류

In [15]:
# 스크래핑한 데이터를 불러오기
df = pd.read_csv('./data/scrapped.csv')

In [27]:
# 데이터프레임의 모양을 확인하고 미리보기
print(df.shape)
df.tail()

(453, 5)


,name,category,rating,ncount,address
448,이때,카페,4.6,22,서울 마포구 동교로38안길 7
449,미소식빵,"제과,베이커리",4.2,2,서울 마포구 동교로27길 92
450,데어데어,"호프,요리주점",3.5,6,서울 마포구 동교로38길 33-9
451,연남고집,술집,4.5,34,서울 마포구 동교로38길 7 지층
452,카페테드,커피전문점,3.4,34,서울 마포구 성미산로32길 51


In [17]:
# Unnamed: 0 칼럼 드랍
df = df.drop('Unnamed: 0', axis = 1)
df.tail()

,name,category,rating,ncount,address
448,이때,카페,4.6,22,서울 마포구 동교로38안길 7
449,미소식빵,"제과,베이커리",4.2,2,서울 마포구 동교로27길 92
450,데어데어,"호프,요리주점",3.5,6,서울 마포구 동교로38길 33-9
451,연남고집,술집,4.5,34,서울 마포구 동교로38길 7 지층
452,카페테드,커피전문점,3.4,34,서울 마포구 성미산로32길 51


In [18]:
# 결측치 확인
df.isna().sum()

name        0
category    1
rating      0
ncount      0
address     0
dtype: int64

In [20]:
# 결측 데이터 확인
df[df['category'].isna()]

,name,category,rating,ncount,address
363,원조시골집할머니네 생선구이백반,NaN,3.2,4,서울 마포구 성미산로 102


In [21]:
# name을 통해 확인한 카테고리로 데이터를 추가
df.loc[df['category'].isna(),'category'] = '백반'
df.isna().sum()

name        0
category    0
rating      0
ncount      0
address     0
dtype: int64

In [22]:
# unique()로 어떤 카테고리의 매장이 수집되었나 확인
df['category'].unique()
# 내가 생각한 '맛집'과 카카오맵이 생각한 '맛집'의 정의가 달라 다양한 카테고리의 매장이 수입되었음
# 나는 '식사를 할 수 있는 곳'을 생각했기 때문에 카페와 술집은 제외하기로 함

array(['육류,고기', '칵테일바', '디저트카페', '일식', '동남아음식', '일식집', '태국음식', '커피전문점',
       '카페', '양식', '회', '중화요리', '제과,베이커리', '해물,생선', '양꼬치', '피자', '이탈리안',
       '멕시칸,브라질', '돈까스,우동', '와인바', '일본식주점', '치킨', '기사식당', '한식', '퓨전요리',
       '샌드위치', '호프,요리주점', '도넛', '갈비', '국밥', '중식', '아시아음식', '일본식라면', '감자탕',
       '삼겹살', '테마카페', '베트남음식', '햄버거', '초밥,롤', '장어', '술집', '곱창,막창', '인도음식',
       '국수', '족발,보쌈', '실내포장마차', '떡볶이', '퓨전한식', '퓨전일식', '참치회', '사주카페',
       '닭요리', '찌개,전골', '분식', '보드카페', '초콜릿', '오뎅바', '백반', '아구', '애견카페',
       '한식뷔페', '패스트푸드', '한정식', '퓨전중식'], dtype=object)

In [31]:
# 카페&디저트 관련 키워드 매장 수 확인
df[df['category'].str.contains('커피|카페|디저트|베이커리|도넛|아이스크림')].shape[0]

173

In [32]:
# 주점관련 키워드 매장 수 확인
df[df['category'].str.contains('포장마차|주점|와인|술집|칵테일|오뎅바')].shape[0]

65

In [34]:
# 각각 is_cafe, is_bar에 인덱스 할당
is_cafe = df[df['category'].str.contains('커피|카페|디저트|베이커리|도넛|아이스크림')].index
is_bar = df[df['category'].str.contains('포장마차|주점|와인|술집|칵테일바|오뎅바')].index

In [35]:
# df에서 드랍하고 데이터 모양 확인
print(df.shape)
df_food = df.drop(is_bar, axis = 0).drop(is_cafe, axis = 0)
print(df_food.shape)
df_food.head()

(453, 5)
(215, 5)


,name,category,rating,ncount,address
0,광고제주정원,"육류,고기",4.3,83,서울 마포구 동교로17길 8
4,미쁘동,일식,3.8,397,서울 마포구 동교로38길 33-21 2층
5,연남토마 본점,일식,3.6,275,서울 마포구 월드컵북로6길 61
6,소이연남,동남아음식,3.9,619,서울 마포구 동교로 267
8,카쿠시타,일식집,4.3,194,서울 마포구 동교로38길 33-15 지하 1층


## 2) 내공있는 맛집을 찾아서


- 포털사이트에서 맛집을 찾아다닌 개인적인 경험을 반영하여 **'내공있는 맛집'** 을 선정했습니다.
- **평점은 '최소한의 기준을 충족하는'** 의 기조로 정했습니다.
    - 제 기준에 만족하는 맛집도 포털사이트 평점이 낮았던 경험이 있었습니다.
    - 맛은 주관적이기에 어떤 맛집도 누군가에겐 1,2 점이 될 수 있기 때문입니다.
    - 평가자의 모수도 다 다르기에 평점순으로 소팅하는 것으로는 맛집을 찾을 수 없었습니다.
    - 최소한의 기준 3.5를 책정하여 분류했습니다.
    
    
- **리뷰 수는 '많을 수록 좋다'** 의 기조로 정했습니다.
    - 평점을 유지하면서 많은 리뷰 수를 쌓는 것은 매우 힘든 일 입니다.
    - 개인적인 경험에서도 평점보다는 많은 리뷰 수를 가지고있는 집이 맛집인 경우가 많았습니다.
    
    
- **'많은 리뷰에 기준 이상의 평점을 가진 집'** 을 **내공있는 맛집** 으로 선정하였습니다.

In [50]:
# 평점(rating) 3.5이상 필터적용, 리뷰 수(ncount) 많은 수대로 소팅하여 상위 10개만 노출
# df_food_naegong에 할당
df_food_naegong = df_food[df_food['rating'] > 3.5].sort_values('ncount', ascending = False).head(10)
df_food_naegong.reset_index(drop = True, inplace = True)
df_food_naegong

,name,category,rating,ncount,address
0,소이연남,동남아음식,3.9,619,서울 마포구 동교로 267
1,경양카츠 연남점,"돈까스,우동",3.9,560,서울 마포구 동교로 238-1 1층
2,연어롭다,"해물,생선",3.6,461,서울 마포구 동교로38길 42 지하1층
3,미쁘동,일식,3.8,397,서울 마포구 동교로38길 33-21 2층
4,툭툭누들타이,태국음식,3.6,355,서울 마포구 성미산로 161-8
5,멕시칸식당베르데,"멕시칸,브라질",4.3,326,서울 마포구 동교로 262-6 1층
6,슬로우캘리,양식,4.0,319,서울 마포구 동교로38길 35
7,다엔조,이탈리안,4.2,288,서울 마포구 연희로1길 59-2 1층
8,발리문,동남아음식,4.4,284,서울 마포구 동교로 248-1 2층
9,연남토마 본점,일식,3.6,275,서울 마포구 월드컵북로6길 61


## 3) 맛집 후 커피 한잔


- 술자리가 아닌 이상 식사후에 갈 수 있는 카페를 같이 알아놓으면 항상 좋았습니다.
- 카카오맵에서 같이 스크래핑된 카페 정보로 맛집 카페를 추천합니다.
- **'내공있는 카페'의 기준** 은 음식점과 동일합니다.
    - 평점 3.5이상, 리뷰 수 상위 10개
    
    
- 식사 후 카페까지 이동거리가 너무 멀면 동행자의 불만족을 유발할 수 있습니다.
    - 카카오 api를 통해 주소의 위경도를 불러옵니다.
    - 먼저 선정한 **'내공있는 맛집'기준, 가장 가까운 '내공있는 카페'** 를 추천합니다.


In [45]:
# 카페관련 키워드로 df_cafe를 생성
df_cafe = df[df['category'].str.contains('커피|카페|디저트|베이커리')]
print(df_cafe.shape)
df_cafe.head()

(170, 7)


,name,category,rating,ncount,address,long,lat
2,광고어니스트팬케이크,디저트카페,4.3,119,서울 마포구 동교로38안길 13,126.924911701123,37.5623712455522
3,땡스오트 연남점,디저트카페,4.1,575,서울 마포구 성미산로23길 68,126.919897441386,37.5662739481646
7,레이어드 연남점,디저트카페,2.8,730,서울 마포구 성미산로 161-4,126.924197234558,37.5649902551936
10,테일러커피 연남1호점,커피전문점,3.8,195,서울 마포구 성미산로 189 1층,126.926397989942,37.5631031597921
11,오랑지,카페,4.7,267,서울 마포구 동교로38길 13 2층,126.925006296467,37.5620587500215


In [ ]:
df_cafe_naegong

In [49]:
# 평점(rating) 3.5이상 필터적용, 리뷰 수(ncount) 많은 수대로 소팅하여 상위 10개만 노출
# df_cafe_naegong에 할당
df_cafe_naegong = df_cafe[df_cafe['rating'] > 3.5].sort_values('ncount', ascending = False).head(10)
df_cafe_naegong.reset_index(drop = True, inplace = True)
df_cafe_naegong

,name,category,rating,ncount,address,long,lat
0,땡스오트 연남점,디저트카페,4.1,575,서울 마포구 성미산로23길 68,126.919897441386,37.5662739481646
1,블룸스토리,카페,3.6,311,서울 마포구 동교로46길 42-20,126.926556651512,37.5618589825992
2,얼스어스,카페,3.9,291,서울 마포구 성미산로 150,126.923271335595,37.5641298365593
3,오랑지,카페,4.7,267,서울 마포구 동교로38길 13 2층,126.925006296467,37.5620587500215
4,코코로카라,"제과,베이커리",3.6,243,서울 마포구 월드컵북로8길 17 1층,126.91847979152,37.5592672811564
5,위아소울메이트,디저트카페,3.7,232,서울 마포구 성미산로 190-11 1층,126.926094899133,37.5625116465661
6,테일러커피 연남1호점,커피전문점,3.8,195,서울 마포구 성미산로 189 1층,126.926397989942,37.5631031597921
7,커피냅로스터스 연남동,카페,4.2,188,서울 마포구 성미산로27길 70,126.920213302301,37.5664284125218
8,씨스루 홍대연남점,디저트카페,4.3,172,서울 마포구 동교로 266-6 1층,126.925583880396,37.5630734571844
9,카페클로이인패리스,디저트카페,4.8,169,서울 마포구 성미산로 190-3 1층,126.926178676706,37.5627193786384


In [51]:
# 라이브러리 추가
import requests
app_key = {'Authorization': 'KakaoAK 8cd51dd1ae22e18082bd9b3b7fc52bbe'}

# 카카오 api를 이용해 위경도 불러오기 함수 선언
def get_long(address):
    global long
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+address
    result = requests.get(url, headers = app_key).json()
    for document in result['documents']:
         long = document['x']
    return long

def get_lat(address):
    global lat
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+address
    result = requests.get(url, headers = app_key).json()
    for document in result['documents']:
         lat = document['y']
    return lat

# 성수 블루보틀주소로 테스트
address = '서울특별시 성동구 아차산로 7 케이티링커스'
get_long(address), get_lat(address)

('127.045641393072', '37.5481060299191')

In [53]:
# food 경도 추가
df_food_naegong['long'] = df_food_naegong['address'].apply(get_long)
# food 위도 추가
df_food_naegong['lat'] = df_food_naegong['address'].apply(get_lat)
df_food_naegong.head()

,name,category,rating,ncount,address,long,lat
0,소이연남,동남아음식,3.9,619,서울 마포구 동교로 267,126.925377878746,37.5635325649477
1,경양카츠 연남점,"돈까스,우동",3.9,560,서울 마포구 동교로 238-1 1층,126.92449833457,37.5611566188805
2,연어롭다,"해물,생선",3.6,461,서울 마포구 동교로38길 42 지하1층,126.925673942632,37.561055822497
3,미쁘동,일식,3.8,397,서울 마포구 동교로38길 33-21 2층,126.926189359347,37.5621018423944
4,툭툭누들타이,태국음식,3.6,355,서울 마포구 성미산로 161-8,126.924044563807,37.5651939623586


In [52]:
# cafe 경도 추가
df_cafe_naegong['long'] = df_cafe_naegong['address'].apply(get_long)
# cafe 위도 추가
df_cafe_naegong['lat'] = df_cafe_naegong['address'].apply(get_lat)
df_cafe_naegong.head()

,name,category,rating,ncount,address,long,lat
0,땡스오트 연남점,디저트카페,4.1,575,서울 마포구 성미산로23길 68,126.919897441386,37.5662739481646
1,블룸스토리,카페,3.6,311,서울 마포구 동교로46길 42-20,126.926556651512,37.5618589825992
2,얼스어스,카페,3.9,291,서울 마포구 성미산로 150,126.923271335595,37.5641298365593
3,오랑지,카페,4.7,267,서울 마포구 동교로38길 13 2층,126.925006296467,37.5620587500215
4,코코로카라,"제과,베이커리",3.6,243,서울 마포구 월드컵북로8길 17 1층,126.91847979152,37.5592672811564


In [54]:
# 카페와 음식점의 유클리드 거리 구하는 함수 선언
def get_distance(cafe_i,food_i):
    # 위도끼리 빼서 x
    x = float(df_cafe_naegong.loc[cafe_i,'lat']) - float(df_food_naegong.loc[food_i,'lat'])
    # 경도끼리 빼서 y
    y = float(df_cafe_naegong.loc[cafe_i,'long']) - float(df_food_naegong.loc[food_i,'long'])
    # 제곱합의 제곱근을 distance에 할당
    distance = np.sqrt(x**2 + y**2)
    return distance

In [63]:
# 테스트 0번 음식맛집과 3번 카페맛집의 거리
print(df_food_naegong.loc[0, 'name'])
print(df_cafe_naegong.loc[3, 'name'])
get_distance(0,3)

소이연남
오랑지


0.007549483305882863

In [111]:
# 결과물로 쓸 df_match생성
df_match = df_food_naegong.copy()

for i in df_food_naegong.index:
    # i음식점으로부터 모든 카페의 거리를 담을 temp 생성
    temp = {}
    for j in df_cafe_naegong.index:  
        # i음식점와 j카페의 거리를 temp에 j키의 벨류로 추가
        temp[j] = get_distance(j,i)
    # temp 딕셔너리 value 중 최소값을 가진 key를 뽑아 best_cafe_index에 할당
    best_cafe_index = min(temp.keys(), key=(lambda k: temp[k]))
    # best_cafe_index로 카페 이름 찾아서 best_cafe_name에 할당
    best_cafe_name = df_cafe_naegong.loc[best_cafe_index,'name']
    print(i, best_cafe_name)
    # df_match에 'closest_best_cafe'로 값 추가
    df_match.loc[i,'closest_best_cafe'] = best_cafe_name

0 씨스루 홍대연남점
1 오랑지
2 블룸스토리
3 위아소울메이트
4 얼스어스
5 씨스루 홍대연남점
6 블룸스토리
7 블룸스토리
8 오랑지
9 코코로카라


In [114]:
# 매칭된 데이터 미리보기
df_match.head()

,name,category,rating,ncount,address,long,lat,closest_best_cafe
0,소이연남,동남아음식,3.9,619,서울 마포구 동교로 267,126.925377878746,37.5635325649477,씨스루 홍대연남점
1,경양카츠 연남점,"돈까스,우동",3.9,560,서울 마포구 동교로 238-1 1층,126.92449833457,37.5611566188805,오랑지
2,연어롭다,"해물,생선",3.6,461,서울 마포구 동교로38길 42 지하1층,126.925673942632,37.561055822497,블룸스토리
3,미쁘동,일식,3.8,397,서울 마포구 동교로38길 33-21 2층,126.926189359347,37.5621018423944,위아소울메이트
4,툭툭누들타이,태국음식,3.6,355,서울 마포구 성미산로 161-8,126.924044563807,37.5651939623586,얼스어스


In [193]:
# 필요한 칼럼은 추가하고 필요없는 칼럼은 버림
# df_cafe_naegong과 조인하여 주소 데이터 추가
df_ngmz = df_match.merge(df_cafe_naegong[['name', 'rating', 'ncount', 'address']],
               left_on = 'closest_best_cafe', right_on = 'name',
               suffixes = ('_food','_cafe')
                # 위경도과 중복된 칼럼은 제거
              ).drop(['long','lat','name_cafe'], axis = 1)

## 4) 결과
- '연남동'지역의 **'내공있는 맛집'** 과 가장 가까운 **'내공있는 카페'** 매칭한 결과물입니다.


In [194]:
df_ngmz

,name_food,category,rating_food,ncount_food,address_food,closest_best_cafe,rating_cafe,ncount_cafe,address_cafe
0,소이연남,동남아음식,3.9,619,서울 마포구 동교로 267,씨스루 홍대연남점,4.3,172,서울 마포구 동교로 266-6 1층
1,멕시칸식당베르데,"멕시칸,브라질",4.3,326,서울 마포구 동교로 262-6 1층,씨스루 홍대연남점,4.3,172,서울 마포구 동교로 266-6 1층
2,경양카츠 연남점,"돈까스,우동",3.9,560,서울 마포구 동교로 238-1 1층,오랑지,4.7,267,서울 마포구 동교로38길 13 2층
3,발리문,동남아음식,4.4,284,서울 마포구 동교로 248-1 2층,오랑지,4.7,267,서울 마포구 동교로38길 13 2층
4,연어롭다,"해물,생선",3.6,461,서울 마포구 동교로38길 42 지하1층,블룸스토리,3.6,311,서울 마포구 동교로46길 42-20
5,슬로우캘리,양식,4.0,319,서울 마포구 동교로38길 35,블룸스토리,3.6,311,서울 마포구 동교로46길 42-20
6,다엔조,이탈리안,4.2,288,서울 마포구 연희로1길 59-2 1층,블룸스토리,3.6,311,서울 마포구 동교로46길 42-20
7,미쁘동,일식,3.8,397,서울 마포구 동교로38길 33-21 2층,위아소울메이트,3.7,232,서울 마포구 성미산로 190-11 1층
8,툭툭누들타이,태국음식,3.6,355,서울 마포구 성미산로 161-8,얼스어스,3.9,291,서울 마포구 성미산로 150
9,연남토마 본점,일식,3.6,275,서울 마포구 월드컵북로6길 61,코코로카라,3.6,243,서울 마포구 월드컵북로8길 17 1층


In [195]:
# csv로 저장
df_ngmz.to_csv('./data/ngmz.csv')

## 5) 테블로 시각화를 위한 전처리
- 크롤링 후 뽑은 내공맛집 데이터를 태블로로 시각화하려고 합니다.
- 태블로에서 시각화하기 좋은 포멧으로 전처리 합니다.

In [196]:
# 사용할 칼럼명 columns에 할당
columns = ['match_id','name','category','rating','ncount','address', 'kind']
columns

['match_id', 'name', 'category', 'rating', 'ncount', 'address', 'kind']

In [197]:
# 음식점 칼럼만 인덱싱하여 df_ngmz_food에 할당
df_ngmz_food = df_ngmz.reset_index().iloc[:,0:6]
# kind에 음식으로 분류
df_ngmz_food['kind'] = 'food'
# 칼럼명 정리
df_ngmz_food.columns = columns
df_ngmz_food

,match_id,name,category,rating,ncount,address,kind
0,0,소이연남,동남아음식,3.9,619,서울 마포구 동교로 267,food
1,1,멕시칸식당베르데,"멕시칸,브라질",4.3,326,서울 마포구 동교로 262-6 1층,food
2,2,경양카츠 연남점,"돈까스,우동",3.9,560,서울 마포구 동교로 238-1 1층,food
3,3,발리문,동남아음식,4.4,284,서울 마포구 동교로 248-1 2층,food
4,4,연어롭다,"해물,생선",3.6,461,서울 마포구 동교로38길 42 지하1층,food
5,5,슬로우캘리,양식,4.0,319,서울 마포구 동교로38길 35,food
6,6,다엔조,이탈리안,4.2,288,서울 마포구 연희로1길 59-2 1층,food
7,7,미쁘동,일식,3.8,397,서울 마포구 동교로38길 33-21 2층,food
8,8,툭툭누들타이,태국음식,3.6,355,서울 마포구 성미산로 161-8,food
9,9,연남토마 본점,일식,3.6,275,서울 마포구 월드컵북로6길 61,food


In [198]:
# 카페 칼럼만 인덱싱하여 df_ngmz_cafe에 할당
df_ngmz_cafe = df_ngmz.reset_index().iloc[:,[0,6,7,8,9]]
# food와 맞춰주기 위해 category와 kind 추가
df_ngmz_cafe['category'] = '카페'
df_ngmz_cafe['kind'] = 'cafe'
# 칼럼 순서 food에 맞춰 정리
df_ngmz_cafe = df_ngmz_cafe[['index', 'closest_best_cafe', 'category', 'rating_cafe', 'ncount_cafe', 'address_cafe','kind']]
# 칼럼명 정리
df_ngmz_cafe.columns = columns
df_ngmz_cafe

,match_id,name,category,rating,ncount,address,kind
0,0,씨스루 홍대연남점,카페,4.3,172,서울 마포구 동교로 266-6 1층,cafe
1,1,씨스루 홍대연남점,카페,4.3,172,서울 마포구 동교로 266-6 1층,cafe
2,2,오랑지,카페,4.7,267,서울 마포구 동교로38길 13 2층,cafe
3,3,오랑지,카페,4.7,267,서울 마포구 동교로38길 13 2층,cafe
4,4,블룸스토리,카페,3.6,311,서울 마포구 동교로46길 42-20,cafe
5,5,블룸스토리,카페,3.6,311,서울 마포구 동교로46길 42-20,cafe
6,6,블룸스토리,카페,3.6,311,서울 마포구 동교로46길 42-20,cafe
7,7,위아소울메이트,카페,3.7,232,서울 마포구 성미산로 190-11 1층,cafe
8,8,얼스어스,카페,3.9,291,서울 마포구 성미산로 150,cafe
9,9,코코로카라,카페,3.6,243,서울 마포구 월드컵북로8길 17 1층,cafe


In [200]:
# 분리하고 정리한 food와 cafe 붙이기
to_tableau = pd.concat([df_ngmz_food,df_ngmz_cafe])
# 지도에 시각화 하기위한 위경도 데이터 추가
to_tableau['long'] = to_tableau['address'].apply(get_long)
to_tableau['lat'] = to_tableau['address'].apply(get_lat)
to_tableau

,match_id,name,category,rating,ncount,address,kind,long,lat
0,0,소이연남,동남아음식,3.9,619,서울 마포구 동교로 267,food,126.925377878746,37.5635325649477
1,1,멕시칸식당베르데,"멕시칸,브라질",4.3,326,서울 마포구 동교로 262-6 1층,food,126.925413866846,37.5628562998941
2,2,경양카츠 연남점,"돈까스,우동",3.9,560,서울 마포구 동교로 238-1 1층,food,126.92449833457,37.5611566188805
3,3,발리문,동남아음식,4.4,284,서울 마포구 동교로 248-1 2층,food,126.924531187219,37.5620246599155
4,4,연어롭다,"해물,생선",3.6,461,서울 마포구 동교로38길 42 지하1층,food,126.925673942632,37.561055822497
5,5,슬로우캘리,양식,4.0,319,서울 마포구 동교로38길 35,food,126.92581263134,37.5613569326774
6,6,다엔조,이탈리안,4.2,288,서울 마포구 연희로1길 59-2 1층,food,126.926485610151,37.5615903513316
7,7,미쁘동,일식,3.8,397,서울 마포구 동교로38길 33-21 2층,food,126.926189359347,37.5621018423944
8,8,툭툭누들타이,태국음식,3.6,355,서울 마포구 성미산로 161-8,food,126.924044563807,37.5651939623586
9,9,연남토마 본점,일식,3.6,275,서울 마포구 월드컵북로6길 61,food,126.921472669429,37.5595119480989


In [201]:
# csv로 저장
to_tableau.to_csv('./data/to_tableau.csv')